In [1]:
from gene.optimisers.crossing import CrossingOptimiser
from ray.tune.schedulers import ASHAScheduler
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ray
import torch
from ray import tune
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms
from tqdm import tqdm
from ray.tune.suggest.bayesopt import BayesOptSearch

from gene.optimisers.annealed_crossing import AnnealedCrossingOptimiser
from gene.targets import get_negative_accuracy_target
from gene.util import get_accuracy

random.seed(0)
torch.manual_seed(0)
np.random.seed(0)

DEVICE = ["cpu", "cuda"][0]
N_EPOCHS = 5

train_data = datasets.MNIST(
    root="./cache",
    download=True,
    train=True,
    transform=transforms.ToTensor()
)
test_data = datasets.MNIST(
    root="./cache",
    download=True,
    train=False,
    transform=transforms.ToTensor()
)
train_data = ray.put(train_data)
test_data = ray.put(test_data)


def train(config):
    # Define the model
    model = torch.nn.Sequential(
        torch.nn.Flatten(),
        torch.nn.Linear(28 * 28, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 10)
    )
    models = [model.to(DEVICE)]

    # Define the optimiser
    optimiser = CrossingOptimiser(target_func=get_negative_accuracy_target,
                                  random_function=lambda shape: torch.normal(0, config["init_std"], shape),
                                  selection_limit=10,
                                  max_couples=10,
                                  n_children_per_couple=2,
                                  device=DEVICE)

    # Define the data loaders
    train_loader = DataLoader(ray.get(train_data), batch_size=1024, shuffle=True)
    test_loader = DataLoader(ray.get(test_data), batch_size=1024)

    for e in range(N_EPOCHS):
        for images, labels in train_loader:
            models = optimiser.step(models, images.to(DEVICE), labels.to(DEVICE))

        tune.report(np.mean([get_accuracy(test_loader, m, DEVICE) for m in models]))


parameters = {"init_std": tune.uniform(0.01, 20)}
search_alg = BayesOptSearch(metric="_metric", mode="max")
analysis = tune.run(
    train,
    search_alg=search_alg,
    scheduler=ASHAScheduler(metric="_metric", mode="max"),
    config=parameters,
    time_budget_s=3600,
    num_samples=-1,
    verbose=0
#     resources_per_trial={'gpu': 1},
)

2022-07-28 12:11:54,565	WARNING function_runner.py:563 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-07-28 12:11:54,576	WARNING bayesopt.py:397 -- BayesOpt does not support specific sampling methods. The Uniform sampler will be dropped.
2022-07-28 12:11:54,623	WARNING tune.py:574 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


(scheduler +4m32s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +4m32s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


2022-07-28 12:16:26,876	INFO bayesopt.py:271 -- Skipping duplicated config: {'init_std': 7.497086866493021}.


(scheduler +6m12s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


2022-07-28 12:18:08,180	INFO bayesopt.py:271 -- Skipping duplicated config: {'init_std': 7.497092572217386}.


...

2022-07-28 13:01:32,649	INFO bayesopt.py:271 -- Skipping duplicated config: {'init_std': 0.1373319214251074}.
2022-07-28 13:01:38,680	INFO bayesopt.py:271 -- Skipping duplicated config: {'init_std': 9.151061151098407}.
2022-07-28 13:02:36,492	INFO bayesopt.py:271 -- Skipping duplicated config: {'init_std': 5.331647268557103}.
2022-07-28 13:03:46,453	INFO bayesopt.py:271 -- Skipping duplicated config: {'init_std': 18.487024643954115}.


(scheduler +53m42s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


2022-07-28 13:06:09,982	INFO bayesopt.py:271 -- Skipping duplicated config: {'init_std': 1.155619398220654}.
2022-07-28 13:06:39,533	INFO bayesopt.py:271 -- Skipping duplicated config: {'init_std': 3.4101057745683034}.


(scheduler +55m52s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


2022-07-28 13:07:53,849	WARNING worker.py:1228 -- The actor or task with ID ffffffffffffffff78fed60ac408e395620cc57101000000 cannot be scheduled right now. You can ignore this message if this Ray cluster is expected to auto-scale or if you specified a runtime_env for this actor or task, which may take time to install.  Otherwise, this is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increasing the resources available to this Ray cluster.
Required resources for this actor or task: {CPU_group_3fbb0565a05b41aac0951f1ffdbaf0ab: 1.000000}, {CPU_group_0_3fbb0565a05b41aac0951f1ffdbaf0ab: 1.000000}
Available resources on this node: {0.000000/8.000000 CPU, 685210800.000000 GiB/685210800.000000 GiB memory, 1.000000/1.000000 GPU, 342605400.000000 GiB/342605400.000000 GiB object_store_memory, 1000.000000/1000.000000 bundle_group_0_05912330f7f004bb74ef448e415d4505, 0.000000/1.000000 CPU_group_0_3fbb0565a05b41aac0951f1ffdbaf0ab, 

In [2]:
analysis_df = analysis.results_df
analysis_df

/home/yaroslav/miniconda3/envs/synapse/lib/python3.8/site-packages/ray/tune/analysis/experiment_analysis.py:565: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


,_metric,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,experiment_tag,config.init_std
trial_id,,,,,,,,,,,,,,,,,,
b4cbf1a6,0.53475,21.844446,True,NaN,NaN,5.0,dbcfb69484774346bc4230125d7b9053,2022-07-28_12-14-07,1.659003e+09,131.225310,18433.0,yaroslav-B365-M-AORUS-ELITE,217.105.36.71,131.225310,0.0,5.0,1_init_std=7.4971,7.497057
b4ebbf68,0.27135,33.228054,True,NaN,NaN,1.0,738ff7723b8a4ffea3966a4d1a3aefb2,2022-07-28_12-12-29,1.659003e+09,33.228054,18431.0,yaroslav-B365-M-AORUS-ELITE,217.105.36.71,33.228054,0.0,1.0,2_init_std=19.015,19.014779
b500773c,0.41193,21.958321,True,NaN,NaN,5.0,a6033c902bfd4e7c973a3daf12877b67,2022-07-28_12-14-06,1.659003e+09,129.855627,18428.0,yaroslav-B365-M-AORUS-ELITE,217.105.36.71,129.855627,0.0,5.0,3_init_std=14.643,14.642559
b515478e,0.24309,34.450798,True,NaN,NaN,1.0,7db7aedf045045a9881286439d3fe2a7,2022-07-28_12-12-31,1.659003e+09,34.450798,18434.0,yaroslav-B365-M-AORUS-ELITE,217.105.36.71,34.450798,0.0,1.0,4_init_std=11.977,11.977183
b530b35c,0.49560,21.790014,True,NaN,NaN,5.0,7afcfd67ed6841318941af1173e5194f,2022-07-28_12-14-08,1.659003e+09,131.416916,18429.0,yaroslav-B365-M-AORUS-ELITE,217.105.36.71,131.416916,0.0,5.0,5_init_std=3.1288,3.128813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0365c884,NaN,NaN,NaN,NaN,NaN,NaN,1aca9fc1aee74e21aaee3b8ed30a635f,2022-07-28_13-11-37,1.659007e+09,NaN,32091.0,yaroslav-B365-M-AORUS-ELITE,217.105.36.71,NaN,NaN,NaN,NaN,14.532354
0ab8c5be,NaN,NaN,NaN,NaN,NaN,NaN,9f62ef6ce760445d85e5d6e3710d196a,2022-07-28_13-11-41,1.659007e+09,NaN,32116.0,yaroslav-B365-M-AORUS-ELITE,217.105.36.71,NaN,NaN,NaN,NaN,6.993179
0d6b24b4,NaN,NaN,NaN,NaN,NaN,NaN,9b830b62dc354d3fa50b0e2666f1b8b9,2022-07-28_13-11-49,1.659007e+09,NaN,32141.0,yaroslav-B365-M-AORUS-ELITE,217.105.36.71,NaN,NaN,NaN,NaN,10.738063


In [3]:
analysis_df.to_csv("result.csv")